In [10]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
max_review_length = 500

In [11]:
import pandas

import collections
df = pandas.read_csv('words_all.csv')
print(df['tag'])
allWords = df['main_text'].str.cat(sep=' ').split(' ')
allTags = list(set(df['tag'].str.cat(sep=' ').split(' ')))

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

vocabularyF, vocabularyR = build_dataset(allWords)
vocab_size = len(vocabularyF)

0                         math
1               implementation
2                         math
3                     geometry
4                         math
5               implementation
6                           dp
7               implementation
8                 numbertheory
9                           dp
10      constructivealgorithms
11              specialproblem
12              implementation
13              specialproblem
14              implementation
15              specialproblem
16              implementation
17              specialproblem
18                     strings
19              specialproblem
20                        math
21              specialproblem
22              implementation
23              specialproblem
24              datastructures
25              implementation
26              specialproblem
27               dfsandsimilar
28              implementation
29              specialproblem
                 ...          
7974                bruteforce
7975    

In [12]:
text = df['main_text'].as_matrix()
data = [[vocabularyF[i] for i in j.split(' ')] for j in text]
labels = [allTags.index(i) for i in df['tag'].as_matrix() if i != ' tag']
train_data = data[:6403]
y_train = np.asarray(labels[:6403])
test_data = data[6403:]
y_test = np.asarray(labels[6403:])
max_review_length = 500
X_train = sequence.pad_sequences(train_data, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_data, maxlen=max_review_length)
y_train

array([11, 27, 11, ..., 26,  8,  0])

In [13]:

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           590048    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 643,349
Trainable params: 643,349
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\drewmarmon\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/3
6403/6403 [==============================] - 27s 4ms/step - loss: 384.1294 - acc: 0.0103
Epoch 2/3
6403/6403 [==============================] - 26s 4ms/step - loss: 382.2923 - acc: 0.0100
Epoch 3/3
6403/6403 [==============================] - 27s 4ms/step - loss: 382.2922 - acc: 0.0100
Accuracy: 0.87%
